- [ ] Combinar esperanza con esperanza norte

- [ ] Resolver problemas con la flota

- [x] Considerar posicion


In [25]:
import datetime as dt
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.backends.backend_pdf import PdfPages as pdf
import seaborn as sns
sns.set_style("darkgrid")

import requests 
import json
import importlib
import re
import xlsxwriter

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sys
import os

ruta_archivo = os.getcwd()
ruta_base =os.path.dirname(ruta_archivo)

## Parametros

In [26]:
zona_sur = ['Andina', 'Antucoya', 'Caserones','Cerro Negro Norte', 'Lomas Bayas', 'Los Pelambres']
zona_norte = ['Radomiro Tomic','Sierra Gorda', 'Gaby', 'Quebrada Blanca', 'Collahuasi',
    'Ministro Hales', 'Chuquicamata', 'SPENCE','Centinela']
todos = zona_norte + zona_sur


In [27]:
# ['Los Pelambres', 'Andina', 'Radomiro Tomic', 'Cerro Negro Norte','Sierra Gorda', 'Gaby', 'Quebrada Blanca', 'Collahuasi', 'Caserones',
# 'Ministro Hales', 'Chuquicamata', 'Antucoya','SPENCE', 'Lomas Bayas', 'Centinela']
faenas = zona_sur
intervalo= 28 #dias

carpeta_resumen = '/../02_Archivos/Resumenes'

decoracion_alerta = False

In [28]:
#  'Visc.40', 'Visc 100'
viscosidad = ['Visc 100']
# 'AL','CR','CU','FE','NI','PB', 'SN'
desgaste = ['AL','CR','CU','FE','NI','PB']
# 'K','NA','SI'
contaminacion = ['K','NA','SI']
# 'B','CA','MG','MO','P','ZN'
aditivos = ['B','CA','MG','MO','P','ZN']
# 'Diluc.','Agua','TBN','Hollin','Oxid.', 'Nitrac.', 'Sulf.','Ind.PQ.'
fisico_quimico = ['Diluc.','Agua','TBN','Hollin','Oxid.', 'Nitrac.', 'Sulf.','Ind.PQ.']

elementos = viscosidad + desgaste + contaminacion + aditivos + fisico_quimico

faenas_centinela = ['Esperanza', 'Esperanza Sur']

In [29]:
today = dt.date.today()
hoy = today.strftime('%Y-%m-%d')
desde = (today- dt.timedelta(intervalo))

In [30]:
guardar_resumen = os.path.abspath(ruta_base + carpeta_resumen)

carpeta_scaa = 'DatosScaa'
carpeta_scaa = os.path.abspath(ruta_base + '/../02_Archivos/' + carpeta_scaa)
carpeta_limites = 'DatosLimites'
carpeta_limites = os.path.abspath(ruta_base + '/../02_Archivos/' + carpeta_limites)

## Datos de Limites

In [31]:
# Leer limites condenatorios
datos_limites = pd.read_csv(carpeta_limites +'\\limites.csv',keep_default_na=False)
limites = datos_limites.copy()


limites['desde'] = pd.to_datetime(limites['desde'])  
limites['hasta'] = pd.to_datetime(limites['hasta']) 

limites = limites[limites.limitecritico.astype(bool)]



## Datos Scaa

In [32]:
datos_scaa = pd.read_csv(carpeta_scaa + '\\consolidado_scaa.csv', index_col='Nro.Muestra')

In [33]:
scaa = datos_scaa.copy()
scaa['Fecha Muestra'] = pd.to_datetime(scaa['Fecha Muestra'])
scaa['Fecha Cambio Aceite'] = pd.to_datetime(scaa['Fecha Muestra'])
scaa['Fecha Analisis'] = pd.to_datetime(scaa['Fecha Analisis'])
filtro_fecha_scaa = (scaa['Fecha Muestra'] > desde.strftime('%Y-%m-%d')) 
scaa = scaa.loc[filtro_fecha_scaa]
scaa = scaa[scaa.Componente == 'Motor Diesel']
scaa['Consideraciones'] = ""
scaa.Faena.replace(faenas_centinela,'Centinela', inplace=True)
scaa.Faena.replace('Qblanca','Quebrada Blanca', inplace=True)

scaa['Equipo'] = scaa['Equipo'].str.replace('.0', '')
scaa['Equipo'] = pd.Series(scaa['Equipo'], dtype="string")

scaa = scaa[['Alerta','Faena','Flota','Laboratorio','Nombre Aceite','Equipo','Posición','Fecha Muestra','Horas Aceite','Fecha Cambio Aceite','Intervalo Cambio de Aceite','Consideraciones'] + elementos]
scaa.sort_values(['Flota','Equipo','Posición','Fecha Muestra'],inplace=True,ascending=[True, True, True,False])


In [34]:
datos_scaa.columns

Index(['Faena', 'Laboratorio', 'Flota', 'Equipo', 'Componente', 'Posición',
       'Fecha Muestra', 'Horas Aceite', 'Fecha Cambio Aceite',
       'Horas Aceite Actual', 'Horometro Cambio Aceite',
       'Intervalo Cambio de Aceite', 'Desviación en mantención',
       'Nombre Aceite', 'Visc.40', 'Visc 100', 'Pto.Infl.', 'Diluc.', 'Agua',
       'TBN', 'TAN', 'Hollin', 'Oxid.', 'Nitrac.', 'Sulf.', 'Ind.PQ.', 'AG',
       'AL', 'B', 'CA', 'CR', 'CU', 'FE', 'K', 'MG', 'MO', 'NA', 'NI', 'P',
       'PB', 'SI', 'SN', 'TI', 'V', 'ZN', 'Respuesta Alerta', 'Alerta',
       'Fecha Analisis'],
      dtype='object')

In [35]:
scaa.Faena.unique()

array(['Quebrada Blanca', 'Cerro Negro Norte', 'Radomiro Tomic',
       'Los Pelambres', 'Collahuasi', 'Andina', 'Caserones',
       'Ministro Hales', 'Gaby', 'Antucoya', 'Sierra Gorda', 'Centinela',
       'Lomas Bayas', 'SPENCE', 'Chuquicamata'], dtype=object)

In [36]:
def conseguir_limites(limites):
    limites_criticos = []
    limites_precaucion = []
    for elemento in elementos:
        temp_limites = limites.copy()[limites.copy().elemento == elemento]
        if temp_limites.empty:
            limites_criticos.append("")
            limites_precaucion.append("")
        else:
            if temp_limites.iloc[temp_limites["desde"].argmax()]['limiteprecaucion'] !='':
                l_critico = float(temp_limites.iloc[temp_limites["desde"].argmax()]['limitecritico'])
                l_precaucion = float(temp_limites.iloc[temp_limites["desde"].argmax()]['limiteprecaucion'])
                limites_criticos.append(round(l_critico,2))
                limites_precaucion.append(round(l_precaucion,2))
    return (limites_criticos, limites_precaucion)

In [37]:
def draw_frame_border(workbook, worksheet, first_row, first_col, rows_count, cols_count,thickness=1):
    if cols_count == 1 and rows_count == 1:
        # whole cell
        worksheet.conditional_format(first_row, first_col,
                                     first_row, first_col,
                                     {'type': 'formula', 'criteria': 'True',
                                     'format': workbook.add_format({'top': thickness, 'bottom':thickness,
                                                                    'left': thickness,'right':thickness})})    
    elif rows_count == 1:
        # left cap
        worksheet.conditional_format(first_row, first_col,
                                 first_row, first_col,
                                 {'type': 'formula', 'criteria': 'True',
                                  'format': workbook.add_format({'top': thickness, 'left': thickness,'bottom':thickness})})
        # top and bottom sides
        worksheet.conditional_format(first_row, first_col + 1,
                                 first_row, first_col + cols_count - 2,
                                 {'type': 'formula', 'criteria': 'True', 'format': workbook.add_format({'top': thickness,'bottom':thickness})})

        # right cap
        worksheet.conditional_format(first_row, first_col+ cols_count - 1,
                                 first_row, first_col+ cols_count - 1,
                                 {'type': 'formula', 'criteria': 'True',
                                  'format': workbook.add_format({'top': thickness, 'right': thickness,'bottom':thickness})})

    elif cols_count == 1:
        # top cap
        worksheet.conditional_format(first_row, first_col,
                                 first_row, first_col,
                                 {'type': 'formula', 'criteria': 'True',
                                  'format': workbook.add_format({'top': thickness, 'left': thickness,'right':thickness})})

        # left and right sides
        worksheet.conditional_format(first_row + 1,              first_col,
                                 first_row + rows_count - 2, first_col,
                                 {'type': 'formula', 'criteria': 'True', 'format': workbook.add_format({'left': thickness,'right':thickness})})

        # bottom cap
        worksheet.conditional_format(first_row + rows_count - 1, first_col,
                                 first_row + rows_count - 1, first_col,
                                 {'type': 'formula', 'criteria': 'True',
                                  'format': workbook.add_format({'bottom': thickness, 'left': thickness,'right':thickness})})

    else:
        # top left corner
        worksheet.conditional_format(first_row, first_col,
                                 first_row, first_col,
                                 {'type': 'formula', 'criteria': 'True',
                                  'format': workbook.add_format({'top': thickness, 'left': thickness})})

        # top right corner
        worksheet.conditional_format(first_row, first_col + cols_count - 1,
                                 first_row, first_col + cols_count - 1,
                                 {'type': 'formula', 'criteria': 'True',
                                  'format': workbook.add_format({'top': thickness, 'right': thickness})})

        # bottom left corner
        worksheet.conditional_format(first_row + rows_count - 1, first_col,
                                 first_row + rows_count - 1, first_col,
                                 {'type': 'formula', 'criteria': 'True',
                                  'format': workbook.add_format({'bottom': thickness, 'left': thickness})})

        # bottom right corner
        worksheet.conditional_format(first_row + rows_count - 1, first_col + cols_count - 1,
                                 first_row + rows_count - 1, first_col + cols_count - 1,
                                 {'type': 'formula', 'criteria': 'True',
                                  'format': workbook.add_format({'bottom': thickness, 'right': thickness})})

        # top
        worksheet.conditional_format(first_row, first_col + 1,
                                     first_row, first_col + cols_count - 2,
                                     {'type': 'formula', 'criteria': 'True', 'format': workbook.add_format({'top': thickness})})

        # left
        worksheet.conditional_format(first_row + 1,              first_col,
                                     first_row + rows_count - 2, first_col,
                                     {'type': 'formula', 'criteria': 'True', 'format': workbook.add_format({'left': thickness})})

        # bottom
        worksheet.conditional_format(first_row + rows_count - 1, first_col + 1,
                                     first_row + rows_count - 1, first_col + cols_count - 2,
                                     {'type': 'formula', 'criteria': 'True', 'format': workbook.add_format({'bottom': thickness})})

        # right
        worksheet.conditional_format(first_row + 1,              first_col + cols_count - 1,
                                     first_row + rows_count - 2, first_col + cols_count - 1,
                                     {'type': 'formula', 'criteria': 'True', 'format': workbook.add_format({'right': thickness})})

In [38]:
def formato_limites_columnas(lp,lc, formato_precaucion,formato_critico, viscosidad,desgaste,contaminacion,aditivos, fisico_quimico_sin_tbn,fila_prec,fila_crit,columna_elemento):
    col_lim = xlsxwriter.utility.xl_col_to_name(columna_elemento)
    pos_lp = '$' + col_lim + '$' +str(fila_prec + 1)
    pos_lc = '$' + col_lim + '$' +str(fila_crit + 1)
    if elemento in viscosidad:
        lp_real = min(lp,lc)
        lc_real = max(lp,lc)
        condicion_prec = {'type':'cell','criteria': '<=','value': pos_lp,'format': formato_critico}
        condicion_crit = {'type':'cell','criteria': '>=','value': pos_lc,'format': formato_critico}
    elif elemento in desgaste + contaminacion + fisico_quimico_sin_tbn:
        lp_real = min(lp,lc)
        lc_real = max(lp,lc)
        condicion_prec = {'type':'cell','criteria': '>=','value': pos_lp,'format': formato_precaucion}
        condicion_crit = {'type':'cell','criteria': '>=','value': pos_lc,'format': formato_critico}
    elif elemento in aditivos + ['TBN']:
        lp_real = max(lp,lc)
        lc_real = min(lp,lc)
        condicion_prec = {'type':'cell','criteria': '<=','value': pos_lp,'format': formato_precaucion}
        condicion_crit = {'type':'cell','criteria': '<=','value': pos_lc,'format': formato_critico}
    return lp_real,lc_real,condicion_prec,condicion_crit

In [39]:
def columnas_unico_valor(scaa_flota, revisar_columnas):
    # Datos que son unicos de la tabla
    titulos = []
    valor = []
    for col in scaa_flota.columns:
        if col in revisar_columnas:
            lista = scaa_flota[col].unique()
            if len(lista) == 1:
                titulos.append(col)
                valor.append(lista[0])
    return titulos, valor

In [40]:
datos_scaa.Flota.unique()

array(['930E-4SE', '930E-4', 'D475A-5E0', 'GD825A-2', '930E-3',
       'PC5500-6DB', 'WD600-6R', 'HD785-7', 'WA500-6', '980E-4', '980E-5',
       'PC5500-6', '930E-5', 'PC5500-6D', 'WA1200-6', 'WA1200-3',
       'WD900-3', 'WA600-6', 'D375A-6R', 'PC600LC-8', '830E-AC',
       'WD600-3', 'D475A-5', 'PC8000-6FSE', '930E-3SE', 'PC8000-6D',
       'PC8000-6E', 'PC300LC-8', 'HD605-7', 'PC300-8', 'WA250-6',
       'PC4000-6BHD', '960E-2K', 'PC8000-6F', 'D375A-5', 'WA470-5',
       'WD900-3A', 'D275AX-5EO', '830E', 'WA500-6R', '730E', 'PC600LC-8E',
       '930E', 'PC450LC-8', 'D475A-8R'], dtype=object)

In [41]:
estilo_critico = {'bg_color': '#EC0000','font_color': '#FFFFFF','bold':True}
estilo_precaucion = {'bg_color': '#FFFF00','font_color': '#000000','bold':True}
estilo_alerta = {'bg_color': '#fff0c7','font_color': '#000000'}
# #E3001B
estilo_cummins = {'bg_color': '#E3001B'}

estilo_cabecera = {'bg_color': '#000000','font_color': '#FFFFFF','bold':True,'num_format': '@'}
estilo_texto = {'num_format': '@'}

In [43]:
# Parametros Base
# Posicion tabla scaa
inicio_col_scaa = 1
inicio_fila_scaa = 5 # min 4

# Tabla Gerenales
gen_espacio_top = 1
gen_espacio_left = 2

# Posicion tabla limites
fila_prec = inicio_fila_scaa-2
fila_crit = inicio_fila_scaa-1

specific_case_930 = '^(?=.*(?:930.*E))(?!.*(?:930.*SE))'

fisico_quimico_sin_tbn = list(fisico_quimico)
fisico_quimico_sin_tbn.remove('TBN')
revision_columnas_unicas = ['Faena','Flota','Laboratorio','Nombre Aceite','Posición']

for faena in faenas:
    limites_faena = limites.copy()[limites.faena == faena]

    

    lista_flotas = limites_faena.flota.unique()
    filtro_flotas = '|'.join([x.replace(" ", ".*") for x in lista_flotas])

    scaa_faena = scaa.copy()[scaa.Faena == faena]
    scaa_faena = scaa_faena[scaa_faena.Flota.str.contains(filtro_flotas)]

    nombre_excel = guardar_resumen+'/Revisión '+faena+" "+hoy+'.xlsx'
    # Crear excel
    writer = pd.ExcelWriter(nombre_excel,engine="xlsxwriter",date_format='dd/mm/yyyy',datetime_format='dd/mm/yyyy')

    workbook  = writer.book

    # estilo por defecto a utilizar
    formato_critico = workbook.add_format(estilo_critico)
    formato_precaucion = workbook.add_format(estilo_precaucion)
    formato_alerta = workbook.add_format(estilo_alerta)
    formato_cabecera = workbook.add_format(estilo_cabecera)
    formato_texto = workbook.add_format(estilo_texto) 

    formato_cummins = workbook.add_format(estilo_cummins)                        

    for flota in lista_flotas:
        # Parametros de hoja
        nombre_hoja = "Reporte " + flota

        

        if flota == '930 E':
            filtro_flota_regex = specific_case_930
        else:
            filtro_flota_regex = flota.replace(" ", ".*")
        # Datos Scaa
        scaa_flota = scaa_faena.copy()[scaa_faena.Flota.str.contains(filtro_flota_regex)]

        if not scaa_flota.empty:
            # Intervalo cambio aceite
            intervalo_aceite = scaa_flota['Intervalo Cambio de Aceite'].copy()
            # scaa_flota.rename(columns = {'Intervalo Cambio de Aceite':'Int. Cambio'}, inplace = True)
            scaa_flota.drop(columns=['Intervalo Cambio de Aceite','Fecha Cambio Aceite','Nombre Aceite'], inplace=True)

            titulos, valor = columnas_unico_valor(scaa_flota, revision_columnas_unicas)
            # Adjunta la tabla de scaa al excel
            # Ordenar los equipos recientemente alertados y ponerlos en el top

            # Generar lista de equipos ordenados
            filtrar_equipo_ultima_alerta = scaa_flota.groupby('Equipo')['Fecha Muestra'].transform('max').eq(scaa_flota['Fecha Muestra'])
            df_equipo_ultima_alerta = scaa_flota[filtrar_equipo_ultima_alerta].sort_values(by=['Alerta'], ascending=[False])

            ultimos_equipos_alertados = df_equipo_ultima_alerta[df_equipo_ultima_alerta.Alerta == 1].sort_values(by=['Fecha Muestra','Equipo'], ascending=[False,True]).Equipo.tolist()
            ultimos_equipos_no_alertados = df_equipo_ultima_alerta[df_equipo_ultima_alerta.Alerta == 0].sort_values(by=['Equipo'], ascending=[True]).Equipo.tolist()
            orden_equipos = ultimos_equipos_alertados + ultimos_equipos_no_alertados
            orden_equipos = list(dict.fromkeys(orden_equipos))

            # ordenar tabla de datos
            scaa_flota['Equipo'] = pd.Categorical(scaa_flota['Equipo'], ordered=True, categories=orden_equipos)
            scaa_flota.sort_values(by=['Equipo','Posición','Fecha Muestra'], ascending=[True,True,False], inplace=True)
            scaa_flota['Equipo'] = pd.Series(scaa_flota['Equipo'], dtype="string")

            #intervalo_equipos = scaa_flota.Equipo.value_counts().sort_index()
            #intervalo_equipos = (scaa_flota.Equipo + scaa_flota.Posición).value_counts().sort_index()
            intervalo_equipos =scaa_flota.groupby(['Equipo','Posición'], sort=False)['Equipo'].count()
            
            scaa_flota = scaa_flota.drop(columns=titulos)

            scaa_flota.to_excel(writer, sheet_name=nombre_hoja,startcol=inicio_col_scaa, startrow=inicio_fila_scaa,index=False)
            
            # Inicializacion variables
            worksheet = writer.sheets[nombre_hoja]
            img_scale = 0.23
            worksheet.insert_image(0,inicio_col_scaa, 'logo_cummins_chile.jpg',{'x_scale':img_scale, 'y_scale': img_scale})
            
            # Posiciones relativas a tabla scaa
            # posiciones SCAA
            scaa_filas = len(scaa_flota)
            scaa_columnas = scaa_flota.columns
            final_col = inicio_col_scaa + len(scaa_columnas) - 1

            # numero de la posicion del elemento inicial en dataframe scaa 
            posicion_inicial = scaa_flota.columns.get_loc(elementos[0])
            # posicion real del elemento inicial en la hoja de excel
            columna_inicial_elementos = inicio_col_scaa + posicion_inicial
            
            # posiciones para el largo de los datos de scaa
            fila_inicial_elemento = inicio_fila_scaa + 1
            fila_final_elemento = inicio_fila_scaa + scaa_filas
            
            # Formato condicional a tabla de scaa
            # Datos Limites
            limites_flota = limites_faena.copy()[limites_faena.flota == flota]
            limites_criticos, limites_precaucion = conseguir_limites(limites_flota)
        
            # tabla limites
            # Titulos de las filas de limite
            worksheet.write(fila_prec,columna_inicial_elementos-1, "Limites Precaución",formato_precaucion)
            worksheet.write(fila_crit,columna_inicial_elementos-1, "Limites Emergencia",formato_critico)
            
            # Valores de tabla de limites + formato condicional de tabla scaa 
            for i, (elemento,lc,lp) in enumerate(list(zip(elementos,limites_criticos, limites_precaucion))):
                # posicion de la columna del elemento en tabla scaa 
                columna_elemento = scaa_flota.columns.get_loc(elemento) + inicio_col_scaa
                # 
                lp_real,lc_real,condicion_prec,condicion_crit = formato_limites_columnas(lp,lc, formato_precaucion,formato_critico, 
                                                                viscosidad,desgaste,contaminacion,aditivos, fisico_quimico_sin_tbn,fila_prec,fila_crit,columna_elemento)
            
                worksheet.write(fila_prec, columna_inicial_elementos+i, lp_real,formato_precaucion)
                worksheet.write(fila_crit,columna_inicial_elementos+i, lc_real,formato_critico)
            
                worksheet.conditional_format(fila_inicial_elemento, columna_elemento, fila_final_elemento, columna_elemento,condicion_crit)
                worksheet.conditional_format(fila_inicial_elemento, columna_elemento, fila_final_elemento, columna_elemento,condicion_prec)

            

            worksheet.autofit()


            
            # tabla datos generales
            for i in range(inicio_fila_scaa):
                titulo_tabla = ""
                valor_tabla = ""
                if i in range(gen_espacio_top,len(titulos)+gen_espacio_top):
                    index = i - gen_espacio_top
                    if titulos[index] != 'Posición':
                        titulo_tabla = titulos[index]
                        valor_tabla = valor[index]
                tabla_espacio_left = inicio_col_scaa + gen_espacio_left
                # titulos
                worksheet.write(i, tabla_espacio_left, titulo_tabla,formato_cabecera)
                # worksheet.merge_range(i+gen_espacio_top, tabla_espacio_left, i+gen_espacio_top, tabla_espacio_left+1, titulos[i],formato_cabecera)
                # valores
                worksheet.write(i, tabla_espacio_left + 1, valor_tabla, formato_texto)
                # worksheet.merge_range(i+gen_espacio_top, tabla_espacio_left + 1, i+gen_espacio_top, tabla_espacio_left+2, valor[i], formato_texto)
                for j in range(inicio_col_scaa,tabla_espacio_left):
                    worksheet.write(i, j, "", formato_cummins)
                

            # Colorea alertas si es 1 == fue alertada por scaa
            if decoracion_alerta:
                posicion_alerta = scaa_flota.columns.get_loc('Alerta')
                for i in range(1,scaa_filas+1):
                    col_alerta = xlsxwriter.utility.xl_col_to_name(posicion_alerta + inicio_col_scaa)
                    worksheet.conditional_format(inicio_fila_scaa + i, inicio_col_scaa, inicio_fila_scaa  + i, final_col, {'type': 'formula',
                                                    'criteria': '=$' + col_alerta + str(i + fila_inicial_elemento) + '=1',
                                                    'format': formato_alerta})

            # crea borde para diferenciar entre los equipos
            counter = fila_inicial_elemento
            for cant in intervalo_equipos:
                draw_frame_border(workbook, worksheet, counter, inicio_col_scaa, cant, final_col,5)
                counter +=cant

            # inmoviliza paneles
            worksheet.freeze_panes(fila_inicial_elemento, columna_inicial_elementos)
    writer.close()

In [ ]:
for j in range(inicio_col_scaa,tabla_espacio_left):
    print(j)

1
2


In [ ]:
tabla_espacio_left

3

In [ ]:
inicio_col_scaa

1

In [ ]:
for i in range(inicio_fila_scaa):
    if i in range(gen_espacio_top,len(titulos)+gen_espacio_top):
        print(i)

1
2
3
4


In [ ]:
fisico_quimico_sin_tbn.remove('TBN')

In [ ]:
len(titulos)+gen_espacio_top

4